In [1]:
import emoji # pip install emoji
import re
import os
import sys
import time
import string
import json
import pandas as pd
import csv
import sklearn

from surprise import SVD
from surprise import accuracy
from surprise.model_selection import KFold
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate

I believe you need the your training graph here

In [2]:
f = open("myfile.json", "r", encoding='utf-8')
sl = json.load(f)
f.close()

In [3]:
all_word_set = {}
for key in sl.keys():
    for word in sl[key].keys():
        if word not in all_word_set:
            all_word_set[word] = 1
        else:
            all_word_set[word] += 1

load in the test dataset(I have generate it for you)

In [4]:
test_df = pd.read_csv('test_rec.csv')

filter the dataset with `all_word_set` and `all_emoji`

In [5]:
def filter_test_data(all_word_set, all_emoji, test_df):
    test_df = test_df.drop(labels = list(test_df[test_df.apply(lambda row:(not row.word in all_word_set) or (not row.emoji in all_emoji),axis=1)].index), axis=0)
    return test_df

In [6]:
test_df = filter_test_data(all_word_set, sl.keys(), test_df)

In [7]:
test_df

,word,emoji
1,"you,",:red_heart_selector:
2,love,:red_heart_selector:
3,hearts,:red_heart_selector:
4,you.,:red_heart_selector:
5,linkedin,:red_heart_selector:
...,...,...
6834,love,:kissing_face_with_closed_eyes:
6838,h,:keycap_7:
6839,k,:keycap_7:
6840,r,:keycap_7:


Evaluation function

In [8]:
def evaluate(test_df, predict_func):
    test_df['res'] = test_df.apply(lambda row:predict_func(row.word, row.emoji) ,axis=1)
    return test_df.mean().res

random predictor

In [9]:
# random predictor
def random_pred(word, emoji):
    global sl # can be removed
    return 1/len(sl)

In [10]:
evaluate(test_df, random_pred)

0.0007639419404124907

trivial neighbor method

In [27]:
# trivial_neighbor predictor
def trivial_neighbor(word, emoji):
    global sl
    total_weight = 0
    good_weight = 0
    for key in sl.keys():
        if word in sl[key].keys():
            total_weight += sl[key][word]
            if key == emoji:
                good_weight += sl[key][word]
    if total_weight == 0:
        return 0
    else:
        return good_weight/total_weight

In [12]:
evaluate(test_df, trivial_neighbor)

0.04321099372389337

## Projection

In [11]:
# GP_MODE:
## 1. simp:simple
## 2. hyper:hyperbolic
## 3. ycn:ycn
## 4. probs:probS
## 5. heats:HeatS
## 6. hybrid:Hybrid
def load_graph(GP_MODE, FG_NAME = './Graphs/bipartite.gpickle'):
    import networkx as nx
    GP_path = './Graphs/Gp_' + GP_MODE + '_10k.gpickle'
    Gp = nx.read_gpickle(GP_path)
    return Gp


In [12]:
Gp1 = load_graph('simp', FG_NAME = './Graphs/bipartite.gpickle')

In [13]:
import networkx as nx
FG = nx.read_gpickle('./Graphs/G.gpickle')

In [18]:
def evaluate_projection(test_df, predict_func):
    test_df['res'] = test_df.apply(lambda row:predict_func(row.word, row.emoji, FG, Gp1) ,axis=1)
    count = 0
    for i in list(test_df['res']):
        if i <=0 :
            count += 1
    print("The ratio with unbipartite emoji is: {}%".format((1-count/len(test_df))*100))        
    return test_df.mean().res

In [16]:
test_df.head()

,word,emoji,res
1,"you,",:red_heart_selector:,0.000764
2,love,:red_heart_selector:,0.000764
3,hearts,:red_heart_selector:,0.000764
4,you.,:red_heart_selector:,0.000764
5,linkedin,:red_heart_selector:,0.000764


In [16]:
# def evaluate_percent(test_df, predict_func):
#     l_w = list(test_df['word'])
#     l_e = list(test_df['emoji'])
#     count = 0
#     for i in range(len(test_df)):
#         if predict_func(l_w[i], l_e[i], FG, Gp1) <= 0:
#             count += 1
        
#     print("The ratio with unbipartite emoji is: {}%".format((1-count/len(test_df))*100))    
#     return (1-count/len(test_df))

In [16]:
import projection_model as pm
# FG, Gp1 = pm.load_graph("Total_FG.p", "Sample_GP.p")

In [18]:
# pm.projection_method(word_in, emoji_in, FG, Gp1)

In [19]:
# simple model
Gp1 = load_graph('simp', FG_NAME = './Graphs/bipartite.gpickle')
evaluate_projection(test_df, pm.projection_method)

The ratio with unbipartite emoji is: 81.86985495883967%


0.05768302189377547

In [20]:
# hyper model
Gp1 = load_graph('hyper', FG_NAME = './Graphs/bipartite.gpickle')
evaluate_projection(test_df, pm.projection_method)

The ratio with unbipartite emoji is: 81.86985495883967%


0.0556077040325089

In [21]:
# ycn model
Gp1 = load_graph('ycn', FG_NAME = './Graphs/bipartite.gpickle')
evaluate_projection(test_df, pm.projection_method)

The ratio with unbipartite emoji is: 81.86985495883967%


0.053371367156108636

In [22]:
# probs model
Gp1 = load_graph('probs', FG_NAME = './Graphs/bipartite.gpickle')
evaluate_projection(test_df, pm.projection_method)

The ratio with unbipartite emoji is: 81.86985495883967%


0.05695721442792921

In [23]:
# heats model
Gp1 = load_graph('heats', FG_NAME = './Graphs/bipartite.gpickle')
evaluate_projection(test_df, pm.projection_method)

The ratio with unbipartite emoji is: 81.86985495883967%


0.060237325612121716

In [24]:
# hybrid model
Gp1 = load_graph('hybrid', FG_NAME = './Graphs/bipartite.gpickle')
evaluate_projection(test_df, pm.projection_method)

The ratio with unbipartite emoji is: 81.86985495883967%


0.058846300030262415

In [ ]:
# GP_MODE:
## 1. simp:simple
## 2. hyper:hyperbolic
## 3. ycn:ycn
## 4. probs:probS
## 5. heats:HeatS
## 6. hybrid:Hybrid

In [16]:
evaluate_projection(test_df, pm.projection_method)

0.043210757225347354

In [ ]:
#0.059

In [15]:
evaluate_projection(test_df, pm.projection_method)

0.01867609078438368

In [19]:
evaluate(test_df, trivial_neighbor)

0.04321099372389337

In [16]:
len(FG.nodes())

20526